# SQL Options in Spark HW

Alirght let's apply what we learned in the lecture to a new dataset!

**But first!**

Let's start with Spark SQL. But first we need to create a Spark Session!

In [1]:
import pyspark
from pyspark.sql import SparkSession

spark=SparkSession.builder.appName('SparkSQL').getOrCreate()
spark

## Read in our DataFrame for this Notebook

For this notebook we will be using the Google Play Store csv file attached to this lecture. Let's go ahead and read it in. 

### About this dataset

Contains a list of Google Play Store Apps and info about the apps like the category, rating, reviews, size, etc. 

**Source:** https://www.kaggle.com/lava18/google-play-store-apps

In [2]:
path = 'Datasets/'

play = spark.read.csv(path+"googleplaystore.csv",header=True,inferSchema=True)

## First things first

Let's check out the first few lines of the dataframe to see what we are working with

In [4]:
play.limit(5).toPandas()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


As well as the schema to make sure all the column types were correctly infered

In [5]:
play.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



Looks like we need to edit some of the datatypes. We need to update Rating, Reviews and Price as integer (float for Rating) values for now, since the Size and Installs variables will need a bit more cleaning. Since we haven't been over this yet, I'm going to provide the code for you here so you can get a quick look at how it used (and how often we need it!).

**make sure to change the df name to whatever you named your df**

In [7]:
from pyspark.sql.types import IntegerType, FloatType
df = play.withColumn("Rating", play["Rating"].cast(FloatType())) \
            .withColumn("Reviews", play["Reviews"].cast(IntegerType())) \
            .withColumn("Price", play["Price"].cast(IntegerType()))
print(df.printSchema())
df.limit(5).toPandas()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: float (nullable = true)
 |-- Reviews: integer (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: integer (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)

None


,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [13]:
df.count()

10841

Looks like that worked! Great! Let's dig in. 

## 1. Create Tempview

Go ahead and create a tempview of the dataframe so we can work with it in spark sql.

In [14]:
df.createOrReplaceTempView('tempview')

## 2. Select all apps with ratings above 4.1

Use your tempview to select all apps with ratings above 4.1

In [16]:
spark.sql("SELECT * FROM tempview WHERE Rating > 4.1").count()

7565

## 3. Now pass your results to an object 
(ie create a spark dataframe)

Select just the App and Rating column where the Category is in the Comic category and the Rating is above 4.5.

In [24]:
spark.sql("SELECT DISTINCT(Category) FROM tempview").limit(36).toPandas()

,Category
0,EVENTS
1,COMICS
2,SPORTS
3,WEATHER
4,VIDEO_PLAYERS
5,AUTO_AND_VEHICLES
6,PARENTING
7,ENTERTAINMENT
8,PERSONALIZATION
9,HEALTH_AND_FITNESS


In [25]:
results = spark.sql("SELECT App,Rating FROM tempview WHERE Rating > 4.5 AND Category = 'COMICS'")

In [26]:
results.limit(5).toPandas()

,App,Rating
0,Manga Master - Best manga & comic reader,4.6
1,GANMA! - All original stories free of charge f...,4.7
2,Röhrich Werner Soundboard,4.7
3,Unicorn Pokez - Color By Number,4.8
4,Manga - read Thai translation,4.6


## 4. Which category has the most cumulative reviews

Only select the one category with the most reivews. 

*Note: will require adding all the review together for each category*

In [ ]:
spark.sql("SELECT App,Rating FROM tempview WHERE Rating > 4.5 AND Category = 'COMICS'")

## 5. Which App has the most reviews?

Display ONLY the top result

Include only the App column and the Reviews column.

In [31]:
spark.sql('SELECT Category, SUM(Reviews) AS reviews_total FROM tempview GROUP BY Category ORDER BY reviews_total DESC LIMIT 1').limit(5).toPandas()

,Category,reviews_total
0,GAME,1585422349


In [61]:
spark.sql('SELECT App, Reviews FROM tempview ORDER BY Reviews DESC LIMIT 1').limit(5).toPandas()

,App,Reviews
0,Facebook,78158306


## 5. Select all apps that contain the word 'dating' anywhere in the title

*Note: we did not cover this in the lecture. You'll have to use your SQL knowledge :) Google it if you need to.*

In [37]:
spark.sql('SELECT * FROM tempview WHERE LOWER(App) LIKE "%dating%"').limit(10).toPandas()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,"Private Dating, Hide App- Blue for PrivacyHider",DATING,NaN,0,18k,100+,Paid,NaN,Everyone,Dating,"July 25, 2017",1.0.1,4.0 and up
1,AMBW Dating App: Asian Men Black Women Interra...,DATING,3.5,2,17M,100+,Paid,NaN,Mature 17+,Dating,"January 21, 2017",1.0.1,4.0 and up
2,Zoosk Dating App: Meet Singles,DATING,4.0,516801,Varies with device,"10,000,000+",Free,0.0,Mature 17+,Dating,"August 2, 2018",Varies with device,Varies with device
3,OkCupid Dating,DATING,4.1,285726,15M,"10,000,000+",Free,0.0,Mature 17+,Dating,"July 30, 2018",11.10.1,4.1 and up
4,Match™ Dating - Meet Singles,DATING,3.7,76646,Varies with device,"10,000,000+",Free,0.0,Mature 17+,Dating,"July 23, 2018",Varies with device,Varies with device
5,"Hily: Dating, Chat, Match, Meet & Hook up",DATING,4.1,2556,56M,"100,000+",Free,0.0,Mature 17+,Dating,"August 1, 2018",2.5.2,4.1 and up
6,Hinge: Dating & Relationships,DATING,4.2,7779,12M,"500,000+",Free,0.0,Mature 17+,Dating,"August 3, 2018",6.1.3,5.0 and up
7,Casual Dating & Adult Singles - Joyride,DATING,4.5,61637,11M,"5,000,000+",Free,0.0,Mature 17+,Dating,"July 31, 2018",4.17.2,4.1 and up
8,BBW Dating & Plus Size Chat,DATING,4.4,12632,29M,"1,000,000+",Free,0.0,Mature 17+,Dating,"July 27, 2018",3.5.0.1,4.1 and up
9,CMB Free Dating App,DATING,4.0,48845,40M,"1,000,000+",Free,0.0,Mature 17+,Dating,"August 1, 2018",4.19.0.2320,4.4 and up


## 6. Use SQL Transformer to display how many free apps there are in this list

In [38]:
from pyspark.ml.feature import SQLTransformer

In [46]:
sqlTrans= SQLTransformer(statement='SELECT * FROM __THIS__ WHERE Type="Free"')


                         

In [48]:
sqlTrans.transform(df).count()

10037

## 7. What is the most popular Genre?

Which genre appears most often in the dataframe. Show only the top result.

In [49]:
from pyspark.sql.functions import expr 

In [58]:
spark.sql("SELECT Genres, COUNT(*) AS number FROM tempview GROUP BY Genres ORDER BY number DESC LIMIT 1").show()

Exception ignored in: <function JavaWrapper.__del__ at 0x7fde6ac05c20>
Traceback (most recent call last):
  File "/Users/antonhaugen/anaconda3/lib/python3.7/site-packages/pyspark/ml/wrapper.py", line 42, in __del__
    if SparkContext._active_spark_context and self._java_obj is not None:
AttributeError: 'SQLTransformer' object has no attribute '_java_obj'


+------+------+
|Genres|number|
+------+------+
| Tools|   842|
+------+------+



## 8. Select all the apps in the 'Tools' genre that have more than 100 reviews

In [60]:
spark.sql("SELECT * FROM tempview WHERE Genres = 'Tools' AND Reviews >100").limit(100).toPandas()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Moto File Manager,TOOLS,4.1,38655,5.9M,"10,000,000+",Free,0,Everyone,Tools,"February 1, 2018",v3.7.93,5.0 and up
1,Google,TOOLS,4.4,8033493,Varies with device,"1,000,000,000+",Free,0,Everyone,Tools,"August 3, 2018",Varies with device,Varies with device
2,Google Translate,TOOLS,4.4,5745093,Varies with device,"500,000,000+",Free,0,Everyone,Tools,"August 4, 2018",Varies with device,Varies with device
3,Moto Display,TOOLS,4.2,18239,Varies with device,"10,000,000+",Free,0,Everyone,Tools,"August 6, 2018",Varies with device,Varies with device
4,Motorola Alert,TOOLS,4.2,24199,3.9M,"50,000,000+",Free,0,Everyone,Tools,"November 21, 2014",1.02.53,4.4 and up
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Network Signal Info,TOOLS,4.1,33926,17M,"1,000,000+",Free,0,Everyone,Tools,"August 3, 2018",4.78.06,4.2 and up
96,Speedtest by Ookla,TOOLS,4.4,1028794,Varies with device,"100,000,000+",Free,0,Everyone,Tools,"July 19, 2018",Varies with device,Varies with device
97,osmino Wi-Fi: free WiFi,TOOLS,4.2,134203,4.1M,"10,000,000+",Free,0,Everyone,Tools,"August 6, 2018",6.06.14,4.4 and up
98,WiFi Overview 360,TOOLS,4.1,15693,18M,"1,000,000+",Free,0,Everyone,Tools,"August 3, 2018",4.32.07,4.2 and up


## That's all folks! Great job!